# Putting it all together - Prepering the environment
You would have gone through most of the elements in this part already, but in case you skipped ahead, and to ensure a clean slate, we will set up our environment from scratch, with a few new elements.
We are going to go through stuff a bit faster, if anyting is unclear, it's recommended to revisit previous sections.


### Setting up the environment for our experiment
Let's begin by importing necessary utilities for displaying agent steps in a human-readable format.

We will now start our  MCP Servers: 
* `mcp-weather` - This server will expose weather-related functionalities that our Llama Stack agent can use as tools.
* `mcp-googlemaps` - this server will allow us to query the google maps api and provide information about locations, businesses and driving directions
* `mcp-parks-info` - This server is a wrapper for our built-in::rag, it will response to sepecific questions about out parks. 

```bash
# MCP-weather configuration
REMOTE_REGISTRY="quay.dev.demo.redhat.com/rhdp/"

podman run -d --name mcp-weather --network=host ${REMOTE_REGISTRY}mcp-weather:latest --port 8005  

# MCP-google-maps configuration

YOUR_API_KEY="AIzaSyAYcxmnVi7ODNOT_A_REAL_KEY_REPLACE_ME"
podman run -d --name mcp-googlemaps-sse --network=host -e GOOGLE_MAPS_API_KEY="ENTER_YOUR_TOKEN" -e MCP_PORT=8006 ${REMOTE_REGISTRY}mcp-googlemaps-sse:latest

# MCP-parks-info

podman run -d --name mcp-parks-info --network=host -e PORT=8007 quay.dev.demo.redhat.com/rhdp/mcp-parks-info:latest


Once the MCP server is running, we can quickly verify its availability by attempting to connect to its `/sse` (Server-Sent Events) endpoint. A successful response indicates the server is live.


In [1]:
!echo Testing MCP-WEATHER ; curl --max-time 1 http://localhost:8005/sse 2>/dev/null
!echo Testing MCP-GOOGLEMAPS-sse ; curl --max-time 1 http://localhost:8006/sse 2>/dev/null 
!echo Testing mcp-parks-info ; curl --max-time 1 http://localhost:8007/sse 2>/dev/null 


Testing MCP-WEATHER
event: endpoint
data: /messages/?session_id=b46073e3773e4cd3add304714383e03a

Testing MCP-GOOGLEMAPS-sse
event: endpoint
data: /message?sessionId=2073da1f-ff0b-4b1d-8c49-fe1696dd9141

Testing mcp-parks-info
event: endpoint
data: /messages/?session_id=e45b99af09c04e42b94894fd359e9b32



### Setting up Llama Stack Agent (Stuff we already know)
Now, let's set up our Llama Stack client. This involves importing necessary libraries, configuring the Llama Stack server URL, and selecting the language model our agent will use.


In [4]:
!pip install -U llama-stack-client==0.2.5 dotenv > /dev/null 2>&1 && echo "pip Python Prerequisites installed succesfuly"
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()
from src.utils import step_printer
# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# These libraries are just here to print the results from the agent in a more human-readable way 
from termcolor import cprint
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
stream=False ## Defaulting to False, you can change this throughout the section to "True" if you wanted to see the output in another format (Using EventLogger)

# for our lab, we will just define our variables manualy here, in a regular application, this would be ready directly from the local .env file and we would comment these lines out
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'

# We will be using the Tavily web search service (docs.tavily.com/)
tavily_search_api_key='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'
provider_data = {"tavily_search_api_key": tavily_search_api_key}

LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")

# List available models and select from allowed models list
allowed_models_list=["meta-llama/Llama-3.2-3B-Instruct"]
#allowed_models_list=["granite3.2:8b"]

selected_model = None


from llama_stack_client import LlamaStackClient
LLAMA_STACK_SERVER='http://localhost:8321'
client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
)

models = client.models.list()

print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")
    # Check if the model identifier contains any of the allowed substrings
    if any(substring in m.identifier for substring in allowed_models_list):
        # Only set selected_model if it hasn't been set yet
        if selected_model is None:
            selected_model = m.identifier
           
# If no allowed model was found, you might want to handle that case
if selected_model is None:
    print("No allowed model found in the list.")


print(f"Selected model (from allowed list): {selected_model}")
            # Removed the break here to show all available models, but the selection logic remains picking the first one


SELECTED_MODEL = selected_model


pip Python Prerequisites installed succesfuly
--- Available models: ---
all-MiniLM-L6-v2 - ollama - all-minilm:latest
granite3.2:8b - ollama - granite3.2:8b
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
Selected model (from allowed list): meta-llama/Llama-3.2-3B-Instruct


### Registering the new MCP Servers as a tools
This is a crucial step: we are now registering our MCP Weather servers as a 'toolgroup' with Llama Stack. This makes the functionalities exposed by them available for our agents to use.

In [13]:
client.toolgroups.register(
        toolgroup_id="mcp::mcp-weather",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":"http://localhost:8005/sse"},
    )

client.toolgroups.register(
        toolgroup_id="mcp::mcp-googlemaps",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":"http://localhost:8006/sse"},
    )
client.toolgroups.register(
        toolgroup_id="mcp::mcp-parks-info",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":"http://localhost:8007/sse"},
    )

## Creating a VectorDB for our RAG to use

In [17]:
#Unregistering vector database in case we are running this over and over again as part of a learning experience

for vector_db_id in client.vector_dbs.list():
    print(f"Unregistering vector database: {vector_db_id.identifier}")
    client.vector_dbs.unregister(vector_db_id=vector_db_id.identifier)

# Select a VectorDB provider from availalbe providers
providers = client.providers.list()
vector_providers = []
for provider in client.providers.list():
    if provider.api == "vector_io":
        #print(f"Found VectorDB provider: {provider.provider_id}\n")  # Simple print
        vector_providers.append(provider)

# In this example, we only have one provider, but on other server we might have many. here, we simply select the first one.
selected_vector_provider = vector_providers[0]

# register our DB
vector_db_id = "Our_Parks_DB"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
    provider_id=selected_vector_provider.provider_id,
)

# Injest documents from local directory (this is so you can test inserting changes easily)
from llama_stack_client.types import Document

files = [
    "Azure_Mongrove_Wilderness.md",
    "Crimson_Basin.md",
    "Granite_Spire.md",
    "Obsidian_Rainforest.md",
    "Prismatic_Painted_Prairie.md",
]

document_dirctory="assets/Parks"

# Read documents into the "documents" array
documents = [
    Document(
        document_id=f"num-{i}",
        content=open(os.path.join(document_dirctory, file), 'r', encoding='utf-8').read(),
        mime_type="text/plain",
        metadata={},
    )
    for i, file in enumerate(files)
]

# Insert the documents into the vectorDB
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=300,
)

print(f"Created {len(documents)} documents from local files into {vector_db_id}")


#print(documents)

Unregistering vector database: Our_Parks_DB
Created 5 documents from local files into Our_Parks_DB


# Summary 


### Useful tools - Remove MCP Servers

In [9]:
toolgroups_to_unregister = [
    "mcp::mcp-weather",
    "mcp::mcp-googlemaps",
    "mcp::mcp-parks-info",
]

for toolgroup_id in toolgroups_to_unregister:
    try:
        print(f"Attempting to unregister toolgroup: {toolgroup_id}")
        client.toolgroups.unregister(toolgroup_id=toolgroup_id)
        print(f"Successfully unregistered toolgroup: {toolgroup_id}")
    except Exception as e:
        print(f"Failed to unregister toolgroup {toolgroup_id}. Error: {e}")

print("\nFinished attempting to unregister toolgroups.")

Attempting to unregister toolgroup: mcp::mcp-weather
Failed to unregister toolgroup mcp::mcp-weather. Error: Error code: 400 - {'detail': "Invalid value: Tool group 'mcp::mcp-weather' not found"}
Attempting to unregister toolgroup: mcp::mcp-googlemaps
Failed to unregister toolgroup mcp::mcp-googlemaps. Error: Error code: 400 - {'detail': "Invalid value: Tool group 'mcp::mcp-googlemaps' not found"}
Attempting to unregister toolgroup: mcp::mcp-parks-info
Failed to unregister toolgroup mcp::mcp-parks-info. Error: Error code: 400 - {'detail': "Invalid value: Tool group 'mcp::mcp-parks-info' not found"}

Finished attempting to unregister toolgroups.
